In [1]:
from subprocess import check_output
import json
import networkx as nx
import matplotlib.pyplot as plt 
from pyvis.network import Network
from Modules.NetVisualizer.net_visualizer import *
from Modules.NetBuilder.net_builder import net_build
import pandas as pd
from random import randint

# DaScra
Uses DaScra code to mine the archive and get a set of stories in a fandom and their information

In [85]:
# story_set_metadata=['node', './Modules/DaScra/scraper.js']
# d = check_output(story_set_metadata)
stories_df= pd.read_excel('./OutputFiles/dascra_output.xlsx',
              dtype={
                  'Additional Tags:': str, 'Archive Warning:': str,
                  'Author:': str, 'Bookmarks:': str,
                  'Category:': str, 'Chapters:': str,
                  'Characters:': str, 'Comments:': str,
                  'Fandom:': str, 'Hits:': int,
                  'Kudos:': int, 'Language:': str,
                  'Rating:': str, 'Relationship:': str,
                  'Series:': str, 'Part:': float,
                  'Source URL:': str, 'Title:': str,
                  'Updated:': str, 'Words:': str,
              }) 


# DO it better all in one pass
additional_tags=','.join([','.join(tags.split(",")) for tags in stories_df["Additional Tags:"]]).replace("/",'*s*').split(',')
print(len(additional_tags))
additional_tags = [item.strip() for item in additional_tags]
additional_tags=list(set(additional_tags))
print(len(additional_tags))

35
30


## Cleaning

In [86]:
stories_df['Additional Tags:']=stories_df['Additional Tags:'].str.split(',')
stories_df['Additional Tags:']=stories_df['Additional Tags:'].apply(lambda x: [item.strip() for item in x])
# stories_df['Additional Tags:']=stories_df['Additional Tags:'].apply(lambda x: ','.join(x))

In [88]:
stories_df['Additional Tags:']

0                                  [Friends to Lovers]
1    [Sophie/Sid, Sophie and Sid, Sid and Sophie, F...
2        [Future Fic, Feelings, Apologies, First Kiss]
3    [Fluff, Smut, cuteness, Making Out, Premature ...
4                              [Domestic Fluff, Humor]
5    [Future Fic, Speculation, Pregnancy, Childbirt...
Name: Additional Tags:, dtype: object

In [89]:
stories_df['Title:']=stories_df['Title:'].apply(lambda x: x.strip())
# stories_df['Title:']


0                               slow dancing
1                              silver lining
2                  why mess up a good thing?
3    ain't that the way love supposed to be?
4                              Empty Nesters
5                          How He Chose Them
Name: Title:, dtype: object

# TagScra
From a set of tags it scrap the RATAS in the archive and create JSON files with the info

In [ ]:
#TODO add the output file name of the JSON tag-data as parameter
args_to_tags_scraper=['node', './Modules/TagScraping/scraper.js' ]


tag_structure_file_name='current_tag_net'
additional_tags=["Hogwarts Eighth Year",'Hogwarts Era',"Hogwarts Fourth Year"]
# additional_tags=["Deaf Character", "Disability","Hogwarts Eighth Year",'Hogwarts Era',"Hogwarts Fourth Year"]


# tag_structure_file_name='Disability_current_tag_net'
# additional_tags=["Disability"]


# tag_structure_file_name='Disability_old_tag_net'
# additional_tags=["Disability"]
# args_to_tags_scraper.extend(['-av','old'])



args_to_tags_scraper.extend(['-of',tag_structure_file_name])
args_to_tags_scraper.extend(['-t'])
args_to_tags_scraper.extend(additional_tags)

# p = check_output(args_to_tags_scraper)

# Building the Network and Visuals

## Current Disability Tag Network

Opens the already mined Disability Tag network
Builds the network
Visualizes it

In [2]:
tag_structure_file_name='Disability_current_tag_net'
additional_tags=["Disability"]

with open('./OutputFiles/'+tag_structure_file_name+'.json', 'r', encoding='utf-8') as f:
  current_data = json.load(f)
f.close()

G = net_build(current_data)

current_dis=net_visualize(G, hierarchical_layout=False, node_sizes=True,headings="Disability Current Tag Network", file_name=tag_structure_file_name)


## OLD Disability Tag Network

Opens the already mined Disability Tag network
Builds the network
Visualizes it

In [3]:
tag_structure_file_name='Disability_old_tag_net'
additional_tags=["Disability"]

with open('./OutputFiles/'+tag_structure_file_name+'.json', 'r', encoding='utf-8') as f:
  old_data = json.load(f)
f.close()

H = net_build(old_data)

# old_dis=net_visualize(H, hierarchical_layout=True, node_sizes=True,headings="Disability Old Tag Network", file_name=tag_structure_file_name)
old_dis=net_visualize(H, hierarchical_layout=False, node_sizes=True,headings="Disability Old Tag Network", file_name=tag_structure_file_name)
# net_visualize(H, hierarchical_layout=False, node_sizes=True,headings="Tag Network of: "+ ",".join(additional_tags), file_name=tag_structure_file_name)


In [ ]:
new_vis, old_vis = get_visuals_older_vs_newest(H.copy(), G.copy(),hierarchical_layout=False)
new_vis.show("disability_current_net.html")
old_vis.show("disability_old_net.html")

## Getting the difference


### Current Disability Tag-Net - Old Disability Tag-Net

Red represents the nodes and edges that will be removed in the difference, the ones that belong to the current verion that also belon to the older

#### Blue represents what it is in the current version that was not in the older version == NEW

In [5]:
diff_G_H,diff_G_H_vis=get_vis_G_diff_H(G,H,False,title="Diff new-old Disability")
diff_G_H_vis.show("diff_new_old.html")

### Old Disability Tag-Net - Current Disability Tag-Net

Red represents the nodes and edges that will be removed in the difference, the ones that belong to the old verion that also belong to the current

#### Blue represents what it was in the Old version that is not there in the current version == CHANGES, EVOLUTION

In [4]:
diff_H_G,diff_H_G_vis=get_vis_G_diff_H(H,G,False, title="Diff old-new Disability")
diff_H_G_vis.show("diff_disability_old_new.html")

# Connecting Stories using the RATAS

In [ ]:
S=nx.Graph()
S.add_nodes_from(stories_df['Title:'])
for story in stories_df['Title:']:
    for conection in stories_df.loc[stories_df['Additional Tags:'].apply(lambda x: 'Fluff' in set(x)), 'Title:']:
        S.add_edge(story,conection) if not story==conection else None

net_visualize(S,False,False,"Full Story-Tag Net",'full_story_net')